In [1]:
import pandas as pd
dataSet = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)
type(dataSet)

pandas.core.frame.DataFrame

In [2]:
dataSet.columns = ['age', 'workclass','fnlwgt','education','education_num','marital_status','occupation','relationship'
              ,'race','sex','capital_gain','capital_loss','hours_per_week','native_country','salary']
dataSet.head()
dataSet['sex'] = dataSet['sex'].str.lstrip() # sremoveing extra spaces from left side 
dataSet['workclass'] = dataSet['workclass'].str.lstrip() #

In [3]:
import sqlite3
connection = sqlite3.connect('sqladb') # Created Database as per guidlines
cursor = connection.cursor() # Cursor to perform task over database

In [5]:
dataSet.to_sql('adultData',connection) # Passing or updating the data set with pandas dataframe

1. Select 10 records from the adult sqladb

In [4]:

q = """SELECT * FROM adultData LIMIT 10;"""
pd.read_sql(q, connection) # Fetching First Ten Records from data base

,index,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


#2. Show me the average hours per week of all men who are working in private sector

In [5]:

q = """SELECT avg(hours_per_week) Average FROM adultData where sex='Male' and workclass='Private'"""
pd.read_sql(q, connection) # Avergae working hours of Private Sector Men

,Average
0,42.221226


#3. Show me the frequency table for education, occupation and relationship, separately

In [7]:
q = """
        create table frequency(
        education varchar(50), 
        occupation varchar(50),
        relationship varchar(50)
        );
    """
pd.read_sql(q, connection)
# Created Seprtate table

In [39]:
q="""select * from frequency"""
data= pd.read_sql(q,connection)


In [34]:
q = """SELECT education, occupation, relationship FROM adultData """

data_= pd.read_sql(q,connection)
data_.head()

,education,occupation,relationship
0,Bachelors,Adm-clerical,Not-in-family
1,Bachelors,Exec-managerial,Husband
2,HS-grad,Handlers-cleaners,Not-in-family
3,11th,Handlers-cleaners,Husband
4,Bachelors,Prof-specialty,Wife


#4. Are there any people who are married, working in private sector and having a masters degree

In [40]:
import sqlite3
from pandas.io import sql
query=''' insert or replace into frequency (education, occupation, relationship) values (?,?,?) '''
connection.executemany(query, data_.to_records(index=False))
q_updated_r ="""select * from frequency"""
Data_update = pd.read_sql(q_updated_r,connection)
Data_update.head()

,education,occupation,relationship
0,Bachelors,Adm-clerical,Not-in-family
1,Bachelors,Exec-managerial,Husband
2,HS-grad,Handlers-cleaners,Not-in-family
3,11th,Handlers-cleaners,Husband
4,Bachelors,Prof-specialty,Wife


In [9]:
#dataSet['marital_status'] = dataSet['marital_status'].str.lstrip()
#dataSet['education'] = dataSet['education'].str.lstrip()

In [41]:
q="""
    select count(*) Result from adultData 
    where 
    (marital_status <>'Never-married' or marital_status <> 'Divorced') and workclass='Private' and education=' Masters'
    """
pd.read_sql(q, connection)

,Result
0,894


#5. What is the average, minimum and maximum age group for people working in different sectors

In [44]:
q="""select workclass,avg(age) Avg,min(age) Min,max(age) max from adultData group by workclass"""
#q="""select count(*) from adultData where workclass='?'"""
workclass = pd.read_sql(q, connection)
workclass.head(10)

,workclass,Avg,Min,max
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


#6. Calculate age distribution by country

In [45]:
q="""select native_country,avg(age) Avg,min(age) Min, max(age) min from adultData group by native_country"""
distribution_by_country= pd.read_sql(q, connection)
distribution_by_country.head()

,native_country,Avg,Min,min
0,?,38.725557,17,90
1,Cambodia,37.789474,18,65
2,Canada,42.545455,17,80
3,China,42.533333,22,75
4,Columbia,39.711864,18,75


#7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [46]:
q="""select capital_gain,capital_loss,(capital_gain+' '+capital_loss) Net_capital_Gain from adultData"""
Net_Capital_Gain = pd.read_sql(q, connection)
Net_Capital_Gain.head()

,capital_gain,capital_loss,Net_capital_Gain
0,2174,0,2174
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


# Task2

### Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

#1. Create an sqlalchemy engine using a sample from the data set

In [47]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sample', echo=True)

In [48]:
conn = engine.connect()

2018-07-27 19:55:33,385 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-27 19:55:33,386 INFO sqlalchemy.engine.base.Engine ()
2018-07-27 19:55:33,387 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-27 19:55:33,387 INFO sqlalchemy.engine.base.Engine ()


In [49]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()
employee = Table('employee', metadata,
              Column('id', Integer, primary_key=True),
              Column('age', Integer),
              Column('workclass', String),
              Column('fnlwgt', Integer),
              Column('education', String),
              Column('education_num', Integer),
              Column('marital_status', String),
              Column('occupation', String),
              Column('relationship', String),
              Column('rasexce', String),
              Column('sex', String),
              Column('capital_gain', Integer),
              Column('capital_loss', Integer),
              Column('hours_per_week', Integer),
              Column('native_country', String),
              Column('salary', String),
)
metadata.create_all(engine) #// view the query

2018-07-27 19:55:36,103 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("employee")
2018-07-27 19:55:36,105 INFO sqlalchemy.engine.base.Engine ()


In [51]:
ins1 = employee.insert().values(id=1, age=39,workclass='State-gov', fnlwgt=77516
                               ,education='Bachelors', education_num=13,marital_status='Never-married', occupation='Adm-clerical'
                            ,relationship='Not-in-family', rasexce='white',sex='Male', capital_gain=2174
                              ,capital_loss=0, hours_per_week=40,native_country='united-states', salary='<=50k'
                              )
ins2 = employee.insert().values(id=2, age=50,workclass='Self-emp-not-inc', fnlwgt=83311
                               ,education='Bachelors', education_num=13,marital_status='Married-civ-spouse', occupation='Exec-managerial'
                              ,relationship='Husband', rasexce='white',sex='Male', capital_gain=0
                              ,capital_loss=0, hours_per_week=13,native_country='united-states', salary='<=50k'
                              )
ins3 = employee.insert().values(id=3, age=38,workclass='Private', fnlwgt=215646
                               ,education='HS-grad', education_num=9,marital_status='Divorced', occupation='Handlers-cleaners'
                              ,relationship='Not-in-family', rasexce='white',sex='Male', capital_gain=0
                              ,capital_loss=0, hours_per_week=40,native_country='united-states', salary='<=50k'
                              )
#str(ins1)
#ins1.compile().params  # to view the data to be inserted
conn.execute(ins1)
conn.execute(ins2)
conn.execute(ins3)

In [52]:
from sqlalchemy.sql import select
s = select([employee])
result = conn.execute(s)
row = result.fetchall()
print(row)

2018-07-27 19:55:54,618 INFO sqlalchemy.engine.base.Engine SELECT employee.id, employee.age, employee.workclass, employee.fnlwgt, employee.education, employee.education_num, employee.marital_status, employee.occupation, employee.relationship, employee.rasexce, employee.sex, employee.capital_gain, employee.capital_loss, employee.hours_per_week, employee.native_country, employee.salary 
FROM employee
2018-07-27 19:55:54,619 INFO sqlalchemy.engine.base.Engine ()
[(1, 39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 'white', 'Male', 2174, 0, 40, 'united-states', '<=50k'), (2, 50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'white', 'Male', 0, 0, 13, 'united-states', '<=50k'), (3, 38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'white', 'Male', 0, 0, 40, 'united-states', '<=50k')]


#2. Write two basic update queries

In [53]:
from sqlalchemy import update

stmt = update(employee).where(employee.c.id==1).values(age=49)
stmt1 = update(employee).where(employee.c.id==1).values(marital_status='Married')
conn.execute(stmt)
conn.execute(stmt1)

2018-07-27 19:55:55,545 INFO sqlalchemy.engine.base.Engine UPDATE employee SET age=? WHERE employee.id = ?
2018-07-27 19:55:55,546 INFO sqlalchemy.engine.base.Engine (49, 1)
2018-07-27 19:55:55,547 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-27 19:55:55,613 INFO sqlalchemy.engine.base.Engine UPDATE employee SET marital_status=? WHERE employee.id = ?
2018-07-27 19:55:55,614 INFO sqlalchemy.engine.base.Engine ('Married', 1)
2018-07-27 19:55:55,625 INFO sqlalchemy.engine.base.Engine COMMIT


In [54]:
s = select([employee])
result = conn.execute(s)
row = result.fetchall()
print(row)

2018-07-27 19:55:56,201 INFO sqlalchemy.engine.base.Engine SELECT employee.id, employee.age, employee.workclass, employee.fnlwgt, employee.education, employee.education_num, employee.marital_status, employee.occupation, employee.relationship, employee.rasexce, employee.sex, employee.capital_gain, employee.capital_loss, employee.hours_per_week, employee.native_country, employee.salary 
FROM employee
2018-07-27 19:55:56,202 INFO sqlalchemy.engine.base.Engine ()
[(1, 49, 'State-gov', 77516, 'Bachelors', 13, 'Married', 'Adm-clerical', 'Not-in-family', 'white', 'Male', 2174, 0, 40, 'united-states', '<=50k'), (2, 50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'white', 'Male', 0, 0, 13, 'united-states', '<=50k'), (3, 38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'white', 'Male', 0, 0, 40, 'united-states', '<=50k')]


#3. Write two delete queries

In [55]:
stmt1=employee.delete().where(employee.c.id==1)
stmt2=employee.delete().where(employee.c.id==2 and employee.c.sex=='Male')
conn.execute(stmt1)
conn.execute(stmt2)

2018-07-27 19:55:57,353 INFO sqlalchemy.engine.base.Engine DELETE FROM employee WHERE employee.id = ?
2018-07-27 19:55:57,354 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-27 19:55:57,355 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-27 19:55:57,441 INFO sqlalchemy.engine.base.Engine DELETE FROM employee WHERE employee.id = ?
2018-07-27 19:55:57,442 INFO sqlalchemy.engine.base.Engine (2,)
2018-07-27 19:55:57,453 INFO sqlalchemy.engine.base.Engine COMMIT


In [56]:
s = select([employee])
result = conn.execute(s)
row = result.fetchall()
print(row)

2018-07-27 19:55:57,905 INFO sqlalchemy.engine.base.Engine SELECT employee.id, employee.age, employee.workclass, employee.fnlwgt, employee.education, employee.education_num, employee.marital_status, employee.occupation, employee.relationship, employee.rasexce, employee.sex, employee.capital_gain, employee.capital_loss, employee.hours_per_week, employee.native_country, employee.salary 
FROM employee
2018-07-27 19:55:57,906 INFO sqlalchemy.engine.base.Engine ()
[(3, 38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'white', 'Male', 0, 0, 40, 'united-states', '<=50k')]


#4. Write two filter queries

In [57]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [58]:
stmt1= session.query(employee).filter(employee.c.id=='Male').first()
stmt2= session.query(employee).filter(employee.c.hours_per_week==45).first()

2018-07-27 19:55:59,284 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-27 19:55:59,285 INFO sqlalchemy.engine.base.Engine SELECT employee.id AS employee_id, employee.age AS employee_age, employee.workclass AS employee_workclass, employee.fnlwgt AS employee_fnlwgt, employee.education AS employee_education, employee.education_num AS employee_education_num, employee.marital_status AS employee_marital_status, employee.occupation AS employee_occupation, employee.relationship AS employee_relationship, employee.rasexce AS employee_rasexce, employee.sex AS employee_sex, employee.capital_gain AS employee_capital_gain, employee.capital_loss AS employee_capital_loss, employee.hours_per_week AS employee_hours_per_week, employee.native_country AS employee_native_country, employee.salary AS employee_salary 
FROM employee 
WHERE employee.id = ?
 LIMIT ? OFFSET ?
2018-07-27 19:55:59,285 INFO sqlalchemy.engine.base.Engine ('Male', 1, 0)
2018-07-27 19:55:59,288 INFO sqlalchemy.engine.base.E

#5. Write two function queries

In [59]:
from sqlalchemy import func
stmt1 = session.query(func.count('*')).select_from(employee).scalar()
stmt2 = session.query(func.count(employee.c.id)).scalar()
print(stmt1)
print(stmt2)

2018-07-27 19:56:00,307 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM employee
2018-07-27 19:56:00,307 INFO sqlalchemy.engine.base.Engine ('*',)
2018-07-27 19:56:00,309 INFO sqlalchemy.engine.base.Engine SELECT count(employee.id) AS count_1 
FROM employee
2018-07-27 19:56:00,309 INFO sqlalchemy.engine.base.Engine ()
1
1
